<a href="https://colab.research.google.com/github/helenatong/oc_p5_Segmentez_des_clients_d_un_site_e-commerce/blob/main/exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

---

## **Prérequis**
- Télécharger [ce fichier TO MOD](https://drive.google.com/file/d/1NWEnq2dD_kOGe-tnFS2w1-C7apQ1ejWx/view?usp=drive_link) dans Google Drive
- Modifier les chemins du fichier (extract_path) dans le code


---

 ## **Contexte et description du notebook**

---

## **Sommaire**

### **I. Installation et mise en place de l'environnement de travail**

### **II. Définitions des fonctions et des variables globales**

### **III. Analyse exploratoire**

# **I. Installation et mise en place de l'environnement de travail**

In [45]:
# importation des librairies
import numpy as np
import pandas as pd
import sqlite3

import zipfile
import os

import re

import sys

import missingno as msno
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer

import datetime

In [46]:
# informations sur l'environnement virtuel
print('Version des librairies :')
print('Python        : ' + sys.version)
print('NumPy         : ' + np.version.full_version)
print('Pandas        : ' + pd.__version__)
print('Matplotlib    : ' + mpl.__version__)
print('Seaborn       : ' + sns.__version__)

now  = datetime.datetime.now().isoformat()
print('Run date      : ' + now)

Version des librairies :
Python        : 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
NumPy         : 1.25.2
Pandas        : 2.0.3
Matplotlib    : 3.7.1
Seaborn       : 0.13.1
Run date      : 2024-06-15T13:23:40.087077


In [47]:
# configurations de l'affichages des donnees
# pd.set_option('display.max_columns', None)

In [48]:
# connexion au drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
# connecter la base de données
conn = sqlite3.connect('/content/drive/MyDrive/openclassrooms/project_list/project_5/data/olist.db')

tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = [table[0] for table in tables]

dfs = {}

for table_name in table_names:
    cursor = conn.execute(f"SELECT * FROM {table_name};")
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    dfs[table_name] = pd.DataFrame(rows, columns=column_names).set_index('index')

    conn.commit()

for table_name, df in dfs.items():
  print(f"Table: {table_name}")

Table: customers
Table: geoloc
Table: order_items
Table: order_pymts
Table: order_reviews
Table: orders
Table: products
Table: sellers
Table: translation


In [50]:
# récupérer les dataframes
df_customers      = dfs.get('customers')
df_geoloc         = dfs.get('geoloc')
df_order_items    = dfs.get('order_items')
df_order_pymts    = dfs.get('order_pymts')
df_order_reviews  = dfs.get('order_reviews')
df_orders         = dfs.get('orders')
df_products       = dfs.get('products')
df_sellers        = dfs.get('sellers')
df_translation    = dfs.get('translation')

# vérifier l'importation
for table_name in dfs.keys():
  df = locals()[f'df_{table_name}']
  print(table_name, '\n', df.head(2), end='\n\n\n')

customers 
                             customer_id                customer_unique_id  \
index                                                                       
0      06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1      18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   

       customer_zip_code_prefix          customer_city customer_state  
index                                                                  
0                         14409                 franca             SP  
1                          9790  sao bernardo do campo             SP  


geoloc 
        geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
index                                                                  
0                             1037       -23.545621       -46.639292   
1                             1046       -23.546081       -46.644820   

      geolocation_city geolocation_state  
index                                     
0    

In [51]:
print(df_translation['product_category_name'].unique().shape)
print(df_products['product_category_name'].unique().shape)

(71,)
(74,)


In [52]:
# créer le dataframe commun
mask = ['customer_id', 'customer_unique_id']
df = df_customers[mask].copy()

df = df.merge(df_orders, how = 'inner', on = ['customer_id'])

mask = ['order_id', 'review_score', 'review_creation_date', 'review_answer_timestamp']
df = df.merge(df_order_reviews[mask], on = ['order_id'], how = 'inner')

df = df.merge(df_order_pymts, on = ['order_id'], how = 'inner')

mask = ['order_id', 'order_item_id', 'product_id']
df = df.merge(df_order_items[mask], on = ['order_id'], how = 'inner')

mask = ['product_id', 'product_category_name']
df = df.merge(df_products[mask], on = ['product_id'], how = 'inner')

df = df.merge(df_translation, on = ['product_category_name'], how = 'left') #s'il manque des traductions

In [59]:
df.dtypes

customer_id                       object
customer_unique_id                object
order_id                          object
order_status                      object
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
review_score                       int64
review_creation_date              object
review_answer_timestamp           object
payment_sequential                 int64
payment_type                      object
payment_installments               int64
payment_value                    float64
order_item_id                      int64
product_id                        object
product_category_name             object
product_category_name_english     object
if_canceled                        int64
dtype: object

In [69]:
df['order_status'].value_counts
df['order_is_canceled'] = df['order_status'].apply(lambda x: 1 if x == 'canceled' else 0)

df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])

df['date_diff'] = (df['order_estimated_delivery_date'] - df['order_delivered_customer_date']).dt.days
df['delivered_late'] = df['date_diff'].apply(lambda x: 1 if x < 0 else 0)
df

,customer_id,customer_unique_id,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_score,...,payment_installments,payment_value,order_item_id,product_id,product_category_name,product_category_name_english,if_canceled,date_diff,order_is_canceled,delivered_late
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,4,...,2,146.87,1,a9516a079e37a9c9c36b9b78b10169e8,moveis_escritorio,office_furniture,0,10.0,0,0
1,8912fc0c3bbf1e2fbf35819e21706718,9eae34bbd3a474ec5d07949ca7de67c0,c1d2b34febe9cd269e378117d6681172,delivered,2017-11-09 00:50:13,2017-11-10 00:47:48,2017-11-22 01:43:37,2017-11-28 00:09:50,2017-12-19,1,...,1,275.79,1,a9516a079e37a9c9c36b9b78b10169e8,moveis_escritorio,office_furniture,0,20.0,0,0
2,8912fc0c3bbf1e2fbf35819e21706718,9eae34bbd3a474ec5d07949ca7de67c0,c1d2b34febe9cd269e378117d6681172,delivered,2017-11-09 00:50:13,2017-11-10 00:47:48,2017-11-22 01:43:37,2017-11-28 00:09:50,2017-12-19,1,...,1,275.79,2,a9516a079e37a9c9c36b9b78b10169e8,moveis_escritorio,office_furniture,0,20.0,0,0
3,f0ac8e5a239118859b1734e1087cbb1f,3c799d181c34d51f6d44bbbc563024db,b1a5d5365d330d10485e0203d54ab9e8,delivered,2017-05-07 20:11:26,2017-05-08 22:22:56,2017-05-19 20:16:31,2017-05-26 09:54:04,2017-06-12,3,...,7,140.61,1,a9516a079e37a9c9c36b9b78b10169e8,moveis_escritorio,office_furniture,0,16.0,0,0
4,6bc8d08963a135220ed6c6d098831f84,23397e992b09769faf5e66f9e171a241,2e604b3614664aa66867856dba7e61b7,delivered,2018-02-03 19:45:40,2018-02-04 22:29:19,2018-02-19 18:21:47,2018-02-28 21:09:00,2018-03-22,4,...,10,137.58,1,a9516a079e37a9c9c36b9b78b10169e8,moveis_escritorio,office_furniture,0,21.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117324,da37711b17efd5f2539e8196ab215f04,5f2971f9805e3ccb030226e30c8e8390,c8203bb57639618630affac9e8e923dd,delivered,2017-03-27 23:04:18,2017-03-27 23:23:38,2017-03-28 14:18:59,2017-03-30 17:06:19,2017-04-13,4,...,1,30.95,1,aea06073397f809424f946979354c9f0,pet_shop,pet_shop,0,13.0,0,0
117325,184e0c2cfc746789643521df0e9ff904,c64ede6d0ae8901b1b6fb03528c1b7e6,45b3000bcd10464ac178f32cd783fc83,delivered,2017-12-07 23:55:46,2017-12-09 23:50:32,2017-12-11 20:29:02,2018-01-05 17:59:38,2018-01-22,4,...,4,304.11,1,bbf975bffd2ae9ee52f513ae5c8a4b27,beleza_saude,health_beauty,0,16.0,0,0
117326,821a7275a08f32975caceff2e08ea262,046470763123d3d6364f89095b4e47ab,49645a8902c1ba980836b7bff991d69f,delivered,2018-04-04 17:50:52,2018-04-04 18:08:41,2018-04-05 16:36:49,2018-04-06 23:32:21,2018-04-18,5,...,3,92.36,1,f6e0a9ce8a6e91c3a0ca2d3005911d20,fashion_bolsas_e_acessorios,fashion_bags_accessories,0,11.0,0,0
117327,1ed0c832c2dd99570a59260e71768bdf,82d46759af0369aad49084bacf85a6c3,51c6d2f460589fa7b65f2da51e860206,delivered,2017-11-14 12:04:09,2017-11-14 12:15:25,2017-11-27 20:44:47,2017-12-19 19:37:33,2017-12-12,1,...,5,167.79,1,c98bf47f7bea8f3aee82fa023786b8a1,eletronicos,electronics,0,-8.0,0,1


In [ ]:
df

In [40]:
df.columns

Index(['customer_id', 'customer_unique_id', 'order_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'review_score', 'review_creation_date',
       'review_answer_timestamp', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'order_item_id', 'product_id',
       'product_category_name', 'product_category_name_english',
       'if_canceled'],
      dtype='object')

In [41]:
# df['diff_estimated_delivered']
# df['tot_cost']

# Recence : last(order purshase)
# Fréquence : compter le nb de order id par client
# Montant : last(dépense par client)

# Feature engineering intéressant:
# retard de livraison order_estimated_delivery_date - order_delivered_customer_date
# score attribué moyen

# voir si granularité a l'item est bien

**DEMARCHE**

Dans la méthode RFM prend en compte la récence, la fréquence et le montant des achats d'un client.

Ici, je vais prendre en compte la date du dernier achat (récence), le nombre de commande passée (fréquence) et le montant total de tous les achats (montant) sur les 2 dernières années (2018-09-03 au 2016-09-04)

In [42]:
print(f"date de la dernière commande passée : {df['order_purchase_timestamp'].max()}")
print(f"date de la première commande passée : {df['order_purchase_timestamp'].min()}")

date de la dernière commande passée : 2018-09-03 09:06:57
date de la première commande passée : 2016-09-04 21:15:19


In [71]:
df = df.groupby(by='customer_unique_id').agg(
        last_order_date=('order_purchase_timestamp', 'min'),
        total_payment=('payment_value', 'sum'),
        mean_review_score=('review_score', 'mean'),
        total_order=('order_id', 'count'),
        nb_order_canceled=('order_is_canceled', 'sum'),
        nb_order_delivered_late=('delivered_late', 'sum'),
        mean_diff_date=('date_diff', 'mean')
    ).reset_index()

In [72]:
df

,customer_unique_id,last_order_date,total_payment,mean_review_score,total_order,nb_order_canceled,nb_order_delivered_late,mean_diff_date
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,141.90,5.0,1,0,0,4.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,27.19,4.0,1,0,0,4.0
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,86.22,3.0,1,0,0,1.0
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,43.62,4.0,1,0,0,11.0
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,196.89,5.0,1,0,0,7.0
...,...,...,...,...,...,...,...,...
94715,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06-08 21:00:36,4134.84,5.0,2,0,0,26.0
94716,fffea47cd6d3cc0a88bd621562a9d061,2017-12-10 20:07:56,84.58,4.0,1,0,0,2.0
94717,ffff371b4d645b6ecea244b27531430a,2017-02-07 15:49:16,112.46,5.0,1,0,0,29.0
94718,ffff5962728ec6157033ef9805bacc48,2018-05-02 15:17:41,133.69,5.0,1,0,0,13.0


# **II. Définitions des fonctions et des variables globales**
